# Noteboook for testing view classification

In [ ]:
import sys
import os
import pandas as pd
import datetime

projectdir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(projectdir, 'src'))

from d00_utils.db_utils import dbReadWriteClassification
from d02_intermediate.download_dcm import s3_download_decomp_dcm
from d02_intermediate.dcm_utils import dcmdir_to_jpgs_for_classification
import d03_classification.predict_view as predict_view

In [ ]:
dcm_dir_suffix = s3_download_decomp_dcm(0.001, 0.5, 'instances_w_labels')

In [ ]:
dcm_dir = os.path.join(os.path.expanduser('~/data/01_raw/'), dcm_dir_suffix)
img_dir = os.path.join(os.path.expanduser('~/data/02_intermediate/'), dcm_dir_suffix)

In [ ]:
dcmdir_to_jpgs_for_classification(dcm_dir, img_dir)

In [ ]:
model_path = '/home/ubuntu/models/view_23_e5_class_11-Mar-2018'
feature_dim = 1

In [ ]:
predictions = predict_view.run_classify(img_dir, 1, model_path)

In [ ]:
label_dim = len(predict_view.view_classes)
model_name = os.path.basename(model_path)
img_dir_basename = os.path.basename(img_dir)

predictions = predict_view.classify(img_dir, feature_dim, label_dim, model_path)

df_columns = ["output_" + x for x in predict_view.view_classes]
df = (
    pd.DataFrame.from_dict(predictions, columns=df_columns, orient="index")
    .rename_axis("file_name")
    .reset_index()
)
df["file_name"] = df["file_name"].apply(lambda x: x.split(".")[0])
df["study_id"] = df["file_name"].apply(lambda x: x.split("_")[1])
df["model_name"] = model_name
df["date_run"] = datetime.datetime.now()
cols = ["study_id", "file_name", "model_name", "date_run"] + df_columns
df = df[cols]

In [ ]:
df.tail()

In [ ]:
io_classification = dbReadWriteClassification()

In [ ]:
io_classification.list_tables()

In [ ]:
io_classification.get_table('predictions')

In [ ]:
io_classification.save_to_db(df[:0], "predictions", if_exists="replace")